# Mandatory Challenge
## Context
You work in the data analysis team of a very important company. On Monday, the company shares some good news with you: you just got hired by a major retail company! So, let's get prepared for a huge amount of work!

Then you get to work with your team and define the following tasks to perform:   
1. You need to start your analysis using data from the past.  
2. You need to define a process that takes your daily data as an input and integrates it.  

You are in charge of the second part, so you are provided with a sample file that you will have to read daily. To complete you task, you need the following aggregates:
* One aggregate per store that adds up the rest of the values.
* One aggregate per item that adds up the rest of the values.

You can import the dataset `retail_sales` from Ironhack's database. 

## Your task
Therefore, your process will consist of the following steps:
1. Read the sample file that a daily process will save in your folder. 
2. Clean up the data.
3. Create the aggregates.
4. Write three tables in your local database: 
    - A table for the cleaned data.
    - A table for the aggregate per store.
    - A table for the aggregate per item.

## Instructions
* Read the csv you can find in Ironhack's database.
* Clean the data and create the aggregates as you consider.
* Create the tables in your local database.
* Populate them with your process.

In [1]:
# your code here
from sqlalchemy import create_engine
import pandas as pd

driver = 'mysql+pymysql:'
user = 'ironhacker_read'
password = 'ir0nhack3r'
ip = '35.239.232.23'
database = 'retail_sales'

engine = f'{driver}//{user}:{password}@{ip}/{database}'

query = """
        SELECT * FROM raw_sales
"""
    
retail = pd.read_sql(query, engine)

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '35.239.232.23' (timed out)")
(Background on this error at: http://sqlalche.me/e/e3q8)

In [13]:
retail.info()
'''So we can see there are no NaN Numbers, so the only thing we can do is to look for outliers,
but I will not do it because the maximum price of an item is 27990 (shown below), and as we have real 
items with values of 19900 we won't say those values are wrong'''

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4545 entries, 0 to 4544
Data columns (total 5 columns):
date            4545 non-null datetime64[ns]
shop_id         4545 non-null int64
item_id         4545 non-null int64
item_price      4545 non-null float64
item_cnt_day    4545 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 177.6 KB


"So we can see there are no NaN Numbers, so the only thing we can do is to look for outliers,\nbut I will not do it because the maximum price of an item is 27990 (shown below), and as we have real \nitems with values of 19900 we won't say those values are wrong"

In [36]:
ag_per_item = retail.drop(["shop_id","date"],1)
'''For this table I'll delete the shop_id column as it's not useful (some items )'''

"For this table I'll delete the shop_id column as it's not useful (some items )"

In [37]:
ag_per_item.head()

,item_id,item_price,item_cnt_day
0,1469,1199.0,1.0
1,21364,479.0,1.0
2,21365,999.0,2.0
3,22104,249.0,2.0
4,22091,179.0,1.0


In [16]:
ag_per_item.min()
#There is a problem with the minimum of the quantity. Let's say it's positive:

item_id         30.0
item_price       3.0
item_cnt_day    -1.0
dtype: float64

In [17]:
ag_per_item.max()
#There is no problem with the maximum of any of them so we will leave it as it is

item_id         22162.0
item_price      27990.0
item_cnt_day       10.0
dtype: float64

In [18]:
ag_per_item["item_cnt_day"] = ag_per_item['item_cnt_day'].apply(lambda x: x if x > 0.0 else -x)
#Negative prices changed

In [19]:
ag_per_item.min()

item_id         30.0
item_price       3.0
item_cnt_day     1.0
dtype: float64

In [24]:
ag_per_item = ag_per_item.groupby(["item_id","item_price"]).sum()
ag_per_item.min()

item_cnt_day    3.0
dtype: float64

In [25]:
ag_per_item.head()
'''So now we have the price and sellings '''

'So now we have the price and sellings '

In [2]:
reven = retail

In [3]:
reven.head()

,date,shop_id,item_id,item_price,item_cnt_day
0,2015-01-04,29,1469,1199.0,1.0
1,2015-01-04,28,21364,479.0,1.0
2,2015-01-04,28,21365,999.0,2.0
3,2015-01-04,28,22104,249.0,2.0
4,2015-01-04,28,22091,179.0,1.0


In [4]:
reven["Revenue"] = reven.item_price * reven.item_cnt_day

In [5]:
reven.head()

,date,shop_id,item_id,item_price,item_cnt_day,Revenue
0,2015-01-04,29,1469,1199.0,1.0,1199.0
1,2015-01-04,28,21364,479.0,1.0,479.0
2,2015-01-04,28,21365,999.0,2.0,1998.0
3,2015-01-04,28,22104,249.0,2.0,498.0
4,2015-01-04,28,22091,179.0,1.0,179.0


In [11]:
reven.groupby('shop_id').sum().head()

,item_id,item_price,item_cnt_day,Revenue
shop_id,,,,
2,966879,99070.5,81.0,103746.0
3,335745,67443.0,33.0,67443.0
4,498624,29361.0,39.0,29361.0
5,620868,33138.0,45.0,33138.0
6,1266894,116352.0,150.0,138678.0


In [42]:
reven.groupby(['shop_id','item_id','item_price']).sum().head(20)

item_cnt_day  Revenue
shop_id item_id item_price                       
2       1970    8999.0               3.0  26997.0
        1971    4499.0               3.0  13497.0
        2871    999.0                3.0   2997.0
        2881    999.0                3.0   2997.0
        3028    2599.0               3.0   7797.0
        5380    3490.0               3.0  10470.0
        10489   399.0                3.0   1197.0
        10826   28.0                 3.0     84.0
        11786   28.0                 3.0     84.0
        11927   539.0                3.0   1617.0
        14256   99.0                 3.0    297.0
        14259   99.0                 3.0    297.0
        14368   28.0                 3.0     84.0
        14716   49.0                 3.0    147.0
        14720   49.0                 3.0    147.0
        15047   1499.0               3.0   4497.0
        15112   159.0                6.0    954.0
        15489   1149.0               3.0   3447.0
        17717   1399.5               6.0   8397.0
        18589   99.0                 3.0    297.0